In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
add = '/home/yishai/Dropbox/CyTOF_Breast/Kaplan_5th/_clusters.csv'


In [2]:
df = pd.read_csv(add).dropna(axis=1, how='all').dropna(axis=0, how='all')
df



,samp,class,clusters
1,4.0,Cycling,1
2,4.0,luminal,rest
3,4.0,basal-like,5
4,4.0,total,8
5,4.1,Cycling,2;5
...,...,...,...
57,20.0,Cycling,3;7
58,20.0,luminal,0;12;9
59,20.0,basal-like,8;1;6
60,20.0,unknown,rest


In [3]:
def get_class(cls,num = True):
    if 'basal' in cls:
        return 0 if num else 'basal'
    elif 'luminal' in cls:
        return 1 if num else 'luminal'
    elif 'Cycling' in cls:
        return 2 if num else 'Cycling'
    elif 'unknown' in cls:
        return 3 if num else 'unknown'

def class_dict(add,num = True):
    dict = {}

    df = pd.read_csv(add).dropna(axis=1, how='all').dropna(axis=0, how='all')

    # find total clusters of each sample and drop row from df 
    s = df['samp'].unique()
    t = [int(df[np.asarray(df['samp']==samp) * np.asarray(df['class']=='total')]['clusters']) for samp in s]
    df = df[df['class']!='total']


    for samp,total_clusters in zip(s,t):#iterate on all samples
        total_clusters = np.arange(total_clusters) # all clusters in sample  
        samp_clusters = []
        for i in df[df['samp']==samp].index:
            if 'rest' not in df.loc[i]['clusters'] : 
                for cluster in np.asarray(df.loc[i]['clusters'].split(';')).astype(int):    
                    dict[(samp,cluster)] = df.loc[i]['class']
                    samp_clusters.append(cluster)
            else:#save index of row with rest info
                ind = i
        for cluster in [t for t in total_clusters if t not in samp_clusters]:#rest clusters
            dict[(samp,cluster)] = df.loc[ind]['class']

    for key in dict.keys():
        dict[key] = get_class(dict[key],num = num)
    return dict
    
dict = class_dict(add = '/home/yishai/Dropbox/CyTOF_Breast/Kaplan_5th/_clusters.csv')


dummy data

df of all samples without classes; sample and cluster info + featurse
data assumed to be normalized

In [4]:
data_size = 10000
features = list(np.arange(20))#generic features for algorithm
# ------------------
# actual data
df = pd.read_csv(add)
df = df.dropna(axis=1, how='all')
df = df.dropna(axis=0, how='all')
samp_nums = [4, 4.1, 5, 7, 7.1, 8, 8.1, 11, 13, 14, 15, 17, 18, 19, 20]
n = len(samp_nums)
samp_total_clusters = [int(df[np.asarray(df['samp']==samp) * np.asarray(df['class']=='total')]['clusters']) for samp in samp_nums]
# ----------
samples = pd.DataFrame(index = np.arange(data_size),columns = ['sample','cluster']+features)#generate data


for i in samples.index:
    ind = i%n
    # samp = 
    samples.loc[i,'sample'] = samp_nums[ind]
    # tot = t[ind]
    samples.loc[i,'cluster'] = np.random.randint(samp_total_clusters[ind], size=1)[0]#random cluster for sample
for f in features:
    samples[f] = np.random.normal(loc = 0,scale = 1,size = samples.shape[0])#random data    

samples_ = samples.copy()

split train test, add class, drop features

In [5]:

def split_into_unique_samples(samples):#split into list of uique i.e 4,4,1,7,8,8.1 etc
    samples_list = []
    for i in samples['sample'].unique():   

        samples_list.append(samples[samples['sample']==i].copy())
    return samples_list
def df_append(arr):
    df = pd.DataFrame(columns = arr[0].columns)#generate data
    for df_ in arr:
        df = pd.concat([df,df_.copy()], ignore_index=True,axis=0,)
    return df
def split_X_y(df):
    y = df['class'].copy()
    X = df.drop('class',axis = 1).copy()
    return X,y
add = '/home/yishai/Dropbox/CyTOF_Breast/Kaplan_5th/_clusters.csv'
dict = class_dict(add)
drop_features = ['sample','cluster']
feature_loc = 0
samples = split_into_unique_samples(samples_.copy())#split into list of uique i.e 4,4,1,7,8,8.1 etc -
for samp in samples: # iterate on uniq sample
    ind0 = samp.index[0]
    sampleID = samp.loc[ind0]['sample']
    # try:
    samp.insert(feature_loc,'class',[int(dict[(sampleID, cluster)]) for  cluster in samp['cluster'].copy()])
    samp.drop(drop_features,axis = 1,inplace = True)
    # except:pass
print(samples[0])

      class         0         1         2         3         4         5  \
0         0  0.528333 -0.474545 -1.276129 -0.152025  0.268125  0.175778   
15        0  1.250816 -0.054959  0.287521  0.579007  1.277651  1.277733   
30        1 -0.532566 -0.838210 -0.525335  0.709498 -1.680021  0.844800   
45        2 -0.260084 -2.456607  0.416544 -1.548655  1.143665 -0.054029   
60        0 -0.523983 -0.043487 -0.883347 -1.743818 -0.421359  0.027362   
...     ...       ...       ...       ...       ...       ...       ...   
9930      0  2.268972  0.991594 -0.613731  1.826613 -0.244207  0.011415   
9945      0 -0.772557  0.725169 -0.460429 -1.405410 -2.106354 -0.560215   
9960      0  0.544713  1.800990 -1.471888 -2.234235 -0.798046 -0.399533   
9975      2 -0.040874 -0.558414 -0.031228 -0.481324 -0.406753 -0.873267   
9990      2 -0.368738  0.459806 -0.104493  0.298066  0.613261  1.160878   

             6         7         8  ...        10        11        12  \
0    -0.544514  1.184763 -

In [6]:
# split data to train and test, merge samples to single df and split  to X and y
from sklearn.model_selection import train_test_split as tts
train, test  = tts(samples,test_size = 0.3,random_state = 42,shuffle = True)#list of df's by samples

X_test,y_test = split_X_y(df_append(test.copy()))
X_train,y_train = split_X_y(df_append(train.copy()))
print(X_train.shape,y_train.shape, X_test.shape,y_test.shape)
print(f'columns: {X_train.columns}')

(6667, 20) (6667,) (3333, 20) (3333,)
columns: Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], dtype='object')


In [7]:
# X_train

convert to pytorch tensor and build dataloader

In [8]:
from torch.utils.data import Dataset, DataLoader
import torch


class ColumnarDataset(Dataset):#return random columns from df in a torch tensor
    def __init__(self, df,y, cats=[]):
        
        # # categorial data - according to categorial feature list (cats)
        # self.dfcats = df[cats] #type: pandas.core.frame.DataFrame
        # self.cats = np.stack([c.values for n, c in self.dfcats.items()], axis=1).astype(np.int64) #tpye: numpy.ndarray
        # continus data
        self.dfconts = df.drop(cats, axis=1) #type: pandas.core.frame.DataFram
        self.X = np.stack([c.values for n, c in self.dfconts.items()], axis=1).astype(np.float32) #tpye: numpy.ndarray
        # y
        self.y = y.values.astype(np.float32)
        
        
    def __len__(self): return len(self.y)

    def __getitem__(self, idx):
        
        # return [self.cats[idx], self.conts[idx], self.y[idx]]
        return [ self.X[idx], self.y[idx]]
params = {'batch_size': 128,
          'shuffle': True,
            'num_workers': 6,
            'drop_last' :True,


          }



train_dataloader = DataLoader(ColumnarDataset(X_train.copy(),  y_train.copy()), **params) #type: torch.utils.data.dataloader.DataLoader
test_dataloader = DataLoader(ColumnarDataset(X_test.copy(),  y_test.copy()), **params) #type: torch.utils.data.dataloader.DataLoader

In [9]:
X,y = next(iter(train_dataloader))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

y

cpu


tensor([0., 2., 2., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 2., 1., 0., 0., 1.,
        0., 3., 1., 1., 0., 2., 0., 3., 2., 1., 1., 0., 1., 2., 2., 1., 1., 1.,
        1., 0., 1., 0., 1., 1., 1., 3., 1., 1., 3., 0., 1., 0., 2., 1., 2., 1.,
        1., 0., 0., 1., 3., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1.,
        2., 2., 2., 1., 2., 2., 3., 0., 0., 2., 1., 0., 0., 2., 1., 1., 0., 1.,
        1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 0., 1., 2., 1., 1., 1., 2., 1., 2., 1., 2., 2., 2., 1.,
        1., 2.])

pytorch tabular example

In [10]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.ssl_models.dae import DenoisingAutoEncoderConfig
from pytorch_tabular import  models

print (f"pytorch_tabular list of models: {[f for f in models.__all__ if 'Config'  in f ]}")


pytorch_tabular list of models: ['CategoryEmbeddingModelConfig', 'NodeConfig', 'TabNetModelConfig', 'MDNConfig', 'AutoIntConfig', 'TabTransformerConfig', 'FTTransformerConfig', 'GatedAdditiveTreeEnsembleConfig']


In [11]:
from pytorch_tabular.models import GatedAdditiveTreeEnsembleConfig
print(GatedAdditiveTreeEnsembleModel)

NameError: name 'GatedAdditiveTreeEnsembleModel' is not defined

In [ ]:


target_name = "Covertype"

cat_col_names = [
    "Wilderness_Area1",
    "Wilderness_Area2",
    "Wilderness_Area3",
    "Wilderness_Area4",
    "Soil_Type1",
    "Soil_Type2",
    "Soil_Type3",
    "Soil_Type4",
    "Soil_Type5",
    "Soil_Type6",
    "Soil_Type7",
    "Soil_Type8",
    "Soil_Type9",
    "Soil_Type10",
    "Soil_Type11",
    "Soil_Type12",
    "Soil_Type13",
    "Soil_Type14",
    "Soil_Type15",
    "Soil_Type16",
    "Soil_Type17",
    "Soil_Type18",
    "Soil_Type19",
    "Soil_Type20",
    "Soil_Type21",
    "Soil_Type22",
    "Soil_Type23",
    "Soil_Type24",
    "Soil_Type25",
    "Soil_Type26",
    "Soil_Type27",
    "Soil_Type28",
    "Soil_Type29",
    "Soil_Type30",
    "Soil_Type31",
    "Soil_Type32",
    "Soil_Type33",
    "Soil_Type34",
    "Soil_Type35",
    "Soil_Type36",
    "Soil_Type37",
    "Soil_Type38",
    "Soil_Type39",
    "Soil_Type40",
]

num_col_names = [
    "Elevation",
    "Aspect",
    "Slope",
    "Horizontal_Distance_To_Hydrology",
    "Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points",
]

feature_columns = num_col_names + cat_col_names + [target_name]
datafile = '/home/yishai/Downloads/covtype.data.gz'

data = pd.read_csv(datafile, header=None, names=feature_columns)
#Dropping NA rows
data.dropna(inplace=True)
# drop categorial features
# data.drop(cat_col_names, axis=1, inplace=True)
# cat_col_names = []
data[cat_col_names].head()

,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data['Wilderness_Area1'	].unique()

array([1, 0])

In [ ]:
# data
target_name

'Covertype'

In [ ]:
data_config = DataConfig(
    target=[target_name],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    normalize_continuous_features=True,#change to false - our case they are normalized
)
trainer_params = {
                'batch_size': 2048,
                'max_epochs' : 10,
                'early_stopping': None,
                'checkpoints': 'valid_loss', # Save best checkpoint monitoring val_loss
                'load_best': True,# After training, load the best checkpoint
                'accelerator': 'auto',# can be 'cpu','gpu', 'tpu', or 'ipu' 
                'devices': -1,# -1 means use all available
                'auto_lr_find': True, # Runs the LRFinder to automatically derive a learning rate

            # 'early_stopping_patience': 5,
            # 'early_stopping_mode': 'min',
                        
}

#default values
optimizer_params = {'optimizer': 'Adam', 'optimizer_params':{},
                    'lr_scheduler': None, 'lr_scheduler_params':{},
                    'lr_scheduler_monitor_metric': "valid_loss"}





model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="2000-1000",
    activation="ReLU",
    dropout=0.1,
    initialization="kaiming",
    head="LinearHead",
    head_config={
        "layers": "",
        "activation": "ReLU",
    },
    learning_rate = 1e-3
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=OptimizerConfig(**optimizer_params),
    trainer_config=TrainerConfig(**trainer_params),
)

2023-11-29 19:11:54,217 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off


In [ ]:
# Splitting data into data for SSL and data for finetuning
ssl, finetune = tts(data, random_state=42, test_size=0.01)
# Train and val splits
ssl_train, ssl_val = tts(ssl, random_state=42)
df_finetune_train, df_finetune_test = tts(finetune, random_state=42)
df_finetune_train, df_finetune_val = tts(df_finetune_train, random_state=42)
print(f"Unlabelled Data: {ssl.shape[0]} rows | Labelled Data: {finetune.shape[0]}")


Unlabelled Data: 575201 rows | Labelled Data: 5811


In [ ]:
tabular_model.fit(
    train=df_finetune_train, 
    validation=df_finetune_val
)
# pytorch_tabular.TabularModel.fit(train, validation=None, test=None, loss=None, metrics=None, metrics_prob_inputs=None, 
#                                  optimizer=None, optimizer_params={}, train_sampler=None, target_transform=None, max_epochs=None, 
#                                  min_epochs=None, seed=42, callbacks=None, datamodule=None, cache_data='memory')

Global seed set to 42
2023-11-29 19:03:30,425 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-11-29 19:03:30,428 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for classification task
2023-11-29 19:03:30,731 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
2023-11-29 19:03:30,999 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
/home/yishai/anaconda3/envs/torch39/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-11-29 19:03:31,060

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  2.2 M │
│ 1 │ _embedding_layer │ Embedding1dLayer          │    276 │
│ 2 │ head             │ LinearHead                │  7.0 K │
│ 3 │ loss             │ CrossEntropyLoss          │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 2.2 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 2.2 M                                                                                                
Total estimated model params size (MB): 8

Output()

`Trainer.fit` stopped: `max_epochs=10` reached.


2023-11-29 19:03:42,192 - {pytorch_tabular.tabular_model:584} - INFO - Training the model completed
2023-11-29 19:03:42,193 - {pytorch_tabular.tabular_model:1258} - INFO - Loading the best model
/home/yishai/anaconda3/envs/torch39/lib/python3.9/site-packages/pytorch_lightning/utilities/cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


In [ ]:
print(1)

1


In [ ]:

import torch.nn as nn

import torch.optim as torch_optim
from torchvision import models
from torch.nn import init
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data
from torch.optim import lr_scheduler

tensor([[ 6.4530e-01, -4.1552e-01, -1.9532e-01,  ..., -8.7123e-01,
         -1.7257e-01, -4.9891e-01],
        [ 7.7081e-01,  4.4592e-02, -1.6168e-01,  ...,  3.5759e-01,
          1.3452e-01, -7.4229e-01],
        [-6.9226e-01, -4.1679e-01,  7.8445e-02,  ...,  1.4433e-03,
          1.1191e+00,  7.8265e-01],
        ...,
        [ 1.0534e+00,  3.0848e-01, -9.8421e-01,  ...,  4.5615e-01,
          1.3803e+00, -1.8561e-01],
        [-7.3249e-01, -1.2989e+00,  3.8116e-01,  ..., -9.1017e-01,
          1.2457e+00, -4.2385e-01],
        [ 1.1645e+00, -1.6287e+00, -2.0924e-01,  ...,  3.9179e-02,
          5.1680e-03,  5.7231e-01]])